In [3]:
from datamodule import CT_Datamodule
from torchvision.transforms import functional as TF
import torch

In [4]:
d = CT_Datamodule("Dataset", num_workers=0)

In [5]:
d.prepare_data()
d.setup("fit")

In [7]:
def calculate_stats(loader: torch.utils.data.DataLoader):
    """
    Calculate the mean and standard deviation of the data in the loader

    Args:
        loader (torch.utils.data.DataLoader): The data loader

    Returns:
        Tuple[torch.Tensor]: The channel-wise mean and standard deviation of the data
    """
    psum = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])
    psum_sq = torch.tensor([0.0] * loader.dataset[0][2][0].shape[0])

    for _, _, inputs, _ in loader:
        inputs = inputs[0]
        psum += inputs.sum(axis=[0, 2, 3])
        psum_sq += (inputs**2).sum(axis=[0, 2, 3])

    count = (
        len(loader.dataset)
        * loader.dataset[0][2][0].shape[1]
        * loader.dataset[0][2][0].shape[2]
    )

    total_mean = psum / count
    total_var = (psum_sq / count) - (total_mean**2)
    total_std = torch.sqrt(total_var)
    return total_mean, total_std

In [8]:
calculate_stats(d.train_dataloader())

(tensor([0.1737, 0.1737, 0.1737]), tensor([0.2584, 0.2584, 0.2584]))

In [6]:
import torch

samples_reg = torch.zeros(4)
samples_tum = torch.zeros(4)

for i, j, _, _ in d.train:
    samples_reg += i
    samples_tum += j

for i, j, _, _ in d.val:
    samples_reg += i
    samples_tum += j

In [11]:
samples_tum

tensor([ 99., 362., 114.,  75.], dtype=torch.float64)

In [2]:
import torch

In [3]:
m= torch.hub.load("facebookresearch/dinov2", "dinov2_vitb14")

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to C:\Users\Owais Makroo/.cache\torch\hub\checkpoints\dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:10<00:00, 32.0MB/s] 


In [4]:
m

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [17]:
m

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [1]:
from model import Classifier

In [2]:
m = Classifier(1e-3, 2e-5, 1)

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [5]:
from loss import SupervisedContrastiveLoss

In [6]:
l = SupervisedContrastiveLoss()

In [7]:
x = torch.randn(32, 16)
x = torch.concat([x.unsqueeze(1), x.unsqueeze(1)], dim=1)
x.shape

torch.Size([32, 2, 16])

In [12]:
l(x)

tensor(0.1657)

In [2]:
from dataset import CT_Dataset

d = CT_Dataset("Dataset")

In [4]:
import numpy as np
regs, ppgls = np.zeros(4), np.zeros(4)

for r, p, _, _ in d:
    regs += r
    ppgls += p

In [9]:
regs

array([305., 201.,  80.,  64.])

In [10]:
ppgls

array([ 99., 362., 114.,  75.])

In [11]:
from datamodule import CT_Datamodule
from model import Classifier
from pytorch_lightning import Trainer
import json

results = {}
data_module = CT_Datamodule("Dataset", batch_size=32, num_workers=5)
data_module.prepare_data()

for i in range(1):
    model = Classifier(lr_dino=1e-5, lr_class=1e-2, weight_decay=5e-4, k=i)
    data_module.set_k(i)

    trainer = Trainer(
        accelerator="gpu",
        max_epochs=10,
        devices=[0],
        log_every_n_steps=1,
    )

    trainer.test(model, data_module)

#     results[i] = trainer.test(model, data_module)[0]
#     print(json.dumps(results, indent=4))

# with open("results.json", "w") as f:
#     json.dump(results, f, indent=4)

Using cache found in C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\Owais Makroo/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` whic

Testing: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined